# Four point functions in the $O(n)$ model

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [2]:
pwd()

"/Users/paulroux/Documents/Recherche/projet_these/code/BootstrapVirasoro/examples"

In [ ]:
using BootstrapVirasoro, BootstrapVirasoro.LoopModels, BenchmarkTools
println("Number of threads: ", Threads.nthreads())

In [ ]:
length(BootstrapVirasoro.cplx_buffer[])

# Bootstrap equations

We solve crossing symmetry equations for the spectrum of the $O(n)$ CFT:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
\end{align}
We then numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(1-x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(1/x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.

In [6]:
const Sig = Channels{Rational}

# Spectrum of O(n) model
c = CC(β=1 / (big"0.8" + big"0.2" * im))
ndiag_indices = [(r, s) for r in 1//2:1//2:20 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
P = big"0.1" + big"0.1"*im
diag_field = Field(c, P=P)
fields = vcat([Field(c, r=r, s=s) for (r, s) in ndiag_indices], Field(c, P=P))

# Determine the parity of the number of legs in 4pt channels
function chan_parities(co::Correlation4)
    V1, V2, V3, V4 = co.fields
    chan_parities = Channels{Rational}((V1.r + V2.r) % 1, (V1.r + V4.r) % 1, (V1.r + V3.r) % 1)
end

# Compute series of blocks for any channel field compatible
# with the correlation co
function LoopSpectra(co, fields, fs)
    Vs = @channels filter(V -> V.r % 1 == chan_parities(co)[chan], fields)
    @channels ChannelSpectrum(co, chan, Vs[chan], fs[chan])
end

# Compute series of blocks, keeping only one of each (field, reflected field) pair
# given indices of the external fields.
function precompute_blocks(indices, fields; parity, precision=10)
    setprecision(BigFloat, floor(Int, 1.2 * precision), base=10)
    Δmax = floor(Int, 1.5 * precision)
    co = Correlation([Field(c, r=r, s=s) for (r, s) in ind], Δmax)
    parity != 0 && (fields = filter(V -> V.diagonal || V.s >= 0, fields))
    fs = Channels(chan -> (V -> IBlock(co, chan, V, parity=parity)))
    LoopSpectra(co, fields, fs)
end

function compute_diagblocks(specs, Ps, chan)
    c = specs.s.corr.fields[1].c
    return [IBlock(specs.s.corr, chan, Field(c, P=P)) for P in Ps]
end

# Solve crossing symmetry for given signature
# Optionnally for many different diagonal blocks in the channel
function solve(
        specs, signature;
        even_spin=Channels(false), 
        rmax=3, show=true, Ps=nothing, diagchan=nothing,
        fix=nothing
    )
    specs = @channels filter(V -> V.diagonal && signature[chan] == 0 || !V.diagonal && V.r >= signature[chan], specs[chan])
    specs = @channels filter(V -> even_spin[chan] ? spin(V) % 2 == 0 : true, specs[chan])
    if Ps === nothing
        res = solve_bootstrap(specs, fix=fix)
        if show
            Base.show(stdout, res.str_cst, rmax=rmax)
        end
    else
        diagblocks = compute_diagblocks(specs, Ps, diagchan)
        res = solve_bootstrap_manyP(specs, diagblocks)
    end
    return res
end

solve (generic function with 1 method)

# Some numerical four-point functions

## $\langle (\frac{1}{2}, 0)^4 \rangle$

In [6]:
ind = ((1//2, 0), (1//2, 0), (1//2, 0), (1//2, 0))
blocks = precompute_blocks(ind, fields, parity=0, precision=20);

In [ ]:
@allocated precompute_blocks(ind, fields, parity=0, precision=20)

### Signature $(0, 1, 1)$

In [37]:
sol = solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false), rmax=2);

 Channel s
┌─────────────────┬───────────────────────────┬────────────────┐
│ Field           │ Structure constant        │ Relative error │
├─────────────────┼───────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │                 1.0+0.0im │            0.0 │
│          (1, 0) │      -0.209496+0.113616im │    8.03089e-18 │
│          (2, 0) │ -0.00301279+0.000871226im │    3.86131e-16 │
│          (2, 1) │  0.00101903-0.000847695im │    1.99533e-16 │
└─────────────────┴───────────────────────────┴────────────────┘
 Channel t
┌────────────┬───────────────────────────┬────────────────┐
│ Field      │ Structure constant        │ Relative error │
├────────────┼───────────────────────────┼────────────────┤
│     (1, 0) │         0.5221+0.127106im │    2.45324e-18 │
│     (1, 1) │       0.20788-0.0478817im │    1.16366e-18 │
│     (2, 0) │ -0.00110538+0.000890279im │    5.09191e-16 │
│ (2, -1//2) │  -0.00107559+0.00116216im │    6.73923e-16 │
│  (2, 1//2) │  -0.00107559+0.00116216

In [38]:
# latex copy-pastable output with backend=:latex keyword argument:
display(c)
display(sol.correlation)
show(stdout, sol.str_cst, rmax=2, backend=:latex)

c = 1.614532871972318339100346020761245674740484429065743944636678200692041522491287 + 2.232249134948096885813148788927335640138408304498269896193771626297577854671372im, β = -1.176470588235294117647058823529411764705882352941176470588235294117647058823537 + 0.2941176470588235294117647058823529411764705882352941176470588235294117647058841im

NonChiralCorrelation{Complex{BigFloat}} with external fields
< (1//2, 0) (1//2, 0) (1//2, 0) (1//2, 0) >

 Channel s
\begin{tabular}{lll}
  \hline
  \textbf{Field} & \textbf{Structure constant} & \textbf{Relative error} \\\hline
  $(P=0.10+0.10im)$ & $1.0$ & 0.0 \\
  $\left(1,0\right)$ & $-0.20949593\;+\;0.113615565\mathrm{i}$ & 8.03089e-18 \\
  $\left(2,0\right)$ & $-0.0030127857\;+\;0.0008712257\mathrm{i}$ & 3.86131e-16 \\
  $\left(2,1\right)$ & $0.0010190255\;-\;0.0008476949\mathrm{i}$ & 1.99533e-16 \\\hline
\end{tabular}
\\ Channel t
\begin{tabular}{lll}
  \hline
  \textbf{Field} & \textbf{Structure constant} & \textbf{Relative error} \\\hline
  $\left(1,0\right)$ & $0.5220995\;+\;0.12710619\mathrm{i}$ & 2.45324e-18 \\
  $\left(1,1\right)$ & $0.2078798\;-\;0.04788174\mathrm{i}$ & 1.16366e-18 \\
  $\left(2,0\right)$ & $-0.0011053841\;+\;0.0008902788\mathrm{i}$ & 5.09191e-16 \\
  $\left(2,\frac{-1}{2}\right)$ & $-0.0010755917\;+\;0.0011621625\mathrm{i}$ & 6.73923e-16 \\
  $\left(2,\frac{1}{2}\right)$ & $-0.0010755917\;+\;0.0011621625\mathrm{i}$ & 5.25469e-16 \\
  $\left(2,1\right)$ & $-0

### Signature $(0, 1, 0)$, $D^{(s)}_{(2, 0)} = 0$

In [5]:
fix = [(:s, diag_field, 1), (:s, Field(c, r=2, s=0), 0)]
solve(blocks, Sig(0, 1, 0), fix=fix, rmax=2);

 Channel s
┌─────────────────┬──────────────────────────┬────────────────┐
│ Field           │ Structure constant       │ Relative error │
├─────────────────┼──────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │                1.0+0.0im │            0.0 │
│          (1, 0) │      -1.16517-0.591022im │    7.65818e-15 │
│          (1, 1) │     0.464566+0.0798147im │     8.4386e-15 │
│          (2, 0) │                0.0+0.0im │            0.0 │
│       (2, 1//2) │ -0.00318409+0.00145081im │    1.01034e-14 │
│      (2, -1//2) │ -0.00318409+0.00145081im │     8.2106e-15 │
│          (2, 1) │  0.00257332-0.00274978im │    6.86828e-15 │
└─────────────────┴──────────────────────────┴────────────────┘
 Channel t
┌────────────┬──────────────────────────┬────────────────┐
│ Field      │ Structure constant       │ Relative error │
├────────────┼──────────────────────────┼────────────────┤
│     (1, 0) │     -0.433576-0.577531im │    1.38597e-14 │
│     (1, 1) │     0.672446+0.0319329i

### Solve for many different Ps

In [6]:
# solve for many different values of the momentum for the diag field in the channel.
# only recompute one column of the linear system each time: very cheap.
Ps = [big"0.8" + big"0.2"*im + i / big(100)*im for i in 1:3];
diagblocks = compute_diagblocks(blocks, Ps, :s)
sol_Ps = solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false),  Ps=Ps, diagchan=:s)
# print one of the results
show(stdout, sol_Ps[2].str_cst, rmax=2)

 Channel s
┌─────────────────┬─────────────────────────┬────────────────┐
│ Field           │ Structure constant      │ Relative error │
├─────────────────┼─────────────────────────┼────────────────┤
│          (1, 0) │     0.253897-0.354829im │    5.69134e-18 │
│ (P=0.80+0.22im) │               1.0+0.0im │            0.0 │
│          (2, 0) │ 0.00488132-0.00418444im │    2.27808e-16 │
│          (2, 1) │ -0.00118272+0.0024477im │    1.13644e-16 │
└─────────────────┴─────────────────────────┴────────────────┘
 Channel t
┌────────────┬─────────────────────────┬────────────────┐
│ Field      │ Structure constant      │ Relative error │
├────────────┼─────────────────────────┼────────────────┤
│     (1, 0) │     -1.08261+0.205966im │     1.5773e-18 │
│     (1, 1) │    -0.347437+0.265879im │     7.9266e-19 │
│     (2, 0) │  0.0047108-0.00153186im │    1.80249e-16 │
│ (2, -1//2) │ 0.00384689-0.00253434im │    2.80274e-16 │
│  (2, 1//2) │ 0.00384689-0.00253434im │    2.39231e-16 │
│     (2, 

Benchmarks: on my 2015, 4-core Intel Macbook pro.
| Precision          | Python | Julia |
|----------|----------|----------|
| $\Delta_{\mathrm{max}}=20$, 13 digits  | 2min23s  | 2.3s  |
| $\Delta_{\mathrm{max}}=30$, 25 digits  | 8min10s  | 3.0s  |
| $\Delta_{\mathrm{max}}=40$, 35 digits  | 23min18s  | 12.6s  |

## $\left\langle P_1 P_2 P_3 P_4 \right\rangle$

In [7]:
ss = rand(Complex{BigFloat}, 4)
ind = Tuple((0, s) for s in ss)
blocks = precompute_blocks(ind, fields, parity=1, precision=15);

In [8]:
sol = solve(blocks, Sig(0, 0, 0), show=true);

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │                  1.0+0.0im │            0.0 │
│          (1, 0) │      -0.043652+0.0461863im │    6.68646e-10 │
│          (1, 1) │     -0.00502903+0.098547im │     1.1386e-10 │
│          (2, 0) │   0.00079105-0.000735744im │     4.28782e-8 │
│       (2, 1//2) │  0.000110591-0.000183974im │     1.46143e-7 │
│          (2, 1) │ -0.000189192-3.69278e-06im │     5.65625e-8 │
│          (3, 0) │                          0 │      0.0607838 │
│       (3, 1//3) │                          0 │      0.0644803 │
│       (3, 2//3) │                          0 │      0.0288652 │
│          (3, 1) │                          0 │      0.0139439 │
└─────────────────┴────────────────────────────┴────────────────┘
 Channel t
┌─────────────────┬───────────────────────────┬───────

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

In [9]:
ind = ((1//2, 0), (1//2, 0), (1, 0), (1, 0))
blocks = precompute_blocks(ind, fields2, parity=1, precision=13);

UndefVarError: UndefVarError: `fields2` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Signature $(0, \frac32, \frac32)$

In [10]:
sol = solve(blocks, Sig(0, 3//2, 3//2));

 Channel s
┌─────────────────┬────────────────────────┬────────────────┐
│ Field           │ Structure constant     │ Relative error │
├─────────────────┼────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │              1.0+0.0im │            0.0 │
│          (1, 0) │    -1.79924+0.421555im │      0.0536187 │
│          (1, 1) │ 0.000273845+0.203333im │       0.179268 │
│          (2, 0) │      1.05837-2.37073im │       0.276612 │
│       (2, 1//2) │   -0.557097+0.386062im │        1.79628 │
│          (2, 1) │    -0.133632-0.76917im │       0.431461 │
│          (3, 0) │      41.9787-27.2899im │       0.626559 │
│       (3, 1//3) │     -34.9665+26.8837im │       0.680484 │
│       (3, 2//3) │       24.938-25.2727im │       0.667133 │
│          (3, 1) │     -11.1594+19.0543im │       0.716201 │
└─────────────────┴────────────────────────┴────────────────┘
 Channel t
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├─

### Signature $(1, \frac12, \frac32)$

In [11]:
solve(blocks, Sig(1, 1//2, 3//2), rmax=2);

 Channel s
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├───────────┼──────────────────────┼────────────────┤
│    (1, 0) │            1.0+0.0im │            0.0 │
│    (1, 1) │ -0.314676-0.227065im │        1.58253 │
│    (2, 0) │   -0.756645+5.0774im │       0.595367 │
│ (2, 1//2) │  -0.515941-3.00656im │       0.911785 │
│    (2, 1) │   0.973783+1.64767im │        1.26323 │
└───────────┴──────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────────┬────────────────┐
│ Field     │ Structure constant     │ Relative error │
├───────────┼────────────────────────┼────────────────┤
│    (1, 0) │ -0.0288501+0.0165005im │          1.011 │
│    (1, 1) │  0.0329273-0.0770832im │       0.777737 │
│    (2, 0) │      3.39492+1.71516im │          2.017 │
│ (2, 1//2) │     -2.78334-1.07935im │        1.75393 │
│    (2, 1) │    1.69856+0.0715347im │        1.71933 │
└───────────┴────────────────────────┴──────

### Signature $(1, \frac32, \frac12)$

In [12]:
solve(blocks, Sig(1, 3//2, 1//2), rmax=2);

 Channel s
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├───────────┼──────────────────────┼────────────────┤
│    (1, 0) │            1.0+0.0im │            0.0 │
│    (1, 1) │ -0.422351-0.268128im │       0.215619 │
│    (2, 0) │    -2.42269+4.0059im │       0.383119 │
│ (2, 1//2) │    1.60603-2.13824im │       0.575963 │
│    (2, 1) │  -0.166614+1.56393im │       0.488147 │
└───────────┴──────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────┬────────────────┐
│ Field     │ Structure constant │ Relative error │
├───────────┼────────────────────┼────────────────┤
│    (2, 0) │  2.02455+1.94047im │       0.673465 │
│ (2, 1//2) │ -1.66763-1.39314im │       0.676533 │
│    (2, 1) │ 1.12007+0.374722im │       0.710549 │
└───────────┴────────────────────┴────────────────┘
 Channel u
┌───────────┬───────────────────────┬────────────────┐
│ Field     │ Structure constant    │ Relative error │
├──────

## $\left\langle (0, \frac12)^3 P_{(1, 0)} \right\rangle$

In [3]:
ind = ((0, 1//2), (0, c.β^2), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, fields, parity=1, precision=20);
sol_diag = solve(blocks, Sig(0, 0, 0), even_spin=Channels(true), rmax=3);

 Channel s
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 1.0+0.0im                 │ 0         │
│ (1, 0)          │ 1.35314+0.621346im        │ 3.2e-18   │
│ (2, 0)          │ -0.0146027+0.00422597im   │ 1.9e-16   │
│ (2, 1)          │ -9.09864e-05+0.00247326im │ 1.3e-16   │
│ (3, 0)          │ 1.58764e-06+1.66872e-06im │ 1.9e-12   │
│ (3, 2//3)       │ 1.18639e-06-4.9787e-07im  │ 1.8e-12   │
└─────────────────┴───────────────────────────┴───────────┘
 Channel t
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 1.0-8.82188e-20im         │ 3.4e-19   │
│ (1, 0)          │ 1.35314+0.621346im        │ 3e-18     │
│ (2, 0)          │ -0.0146027+0.00422597im   │ 9.6e-17   │
│ (2, 1)          

## $\left\langle (0, \frac12)^3 (1, 0) \right\rangle$

In [4]:
ind = ((0, 1//2), (1, 0), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, fields, parity=1, precision=20);

In [5]:
sol1 = solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false),show=true, rmax=2);
sol2 = solve(blocks, Sig(1, 0, 1), even_spin=Channels(false, true, false), show=false);
sol3 = solve(blocks, Sig(1, 1, 0), even_spin=Channels(false, false, true), show=false);

 Channel s
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 1.0+0.0im                 │ 0         │
│ (1, 0)          │ 0                         │ 0.31      │
│ (2, 0)          │ -0.00585115-0.000436412im │ 1.5e-17   │
│ (2, 1)          │ 0.00227685-0.000981606im  │ 1.5e-17   │
└─────────────────┴───────────────────────────┴───────────┘
 Channel t
┌───────────┬──────────────────────────┬───────────┐
│ Field     │ Structure constant       │ Rel. err. │
├───────────┼──────────────────────────┼───────────┤
│ (1, 0)    │ 0.676571+0.310673im      │ 8.6e-20   │
│ (1, 1)    │ 0.286591-0.0446828im     │ 4.8e-20   │
│ (2, 0)    │ -0.00437577+0.00233119im │ 1e-17     │
│ (2, 1//2) │ -0.00327768+0.00228679im │ 9.9e-18   │
│ (2, 1)    │ -0.00118392+0.00172743im │ 8.3e-18   │
└───────────┴──────────────────────────┴───────────┘
 Channel u
┌─────────

In [6]:
show(
    stdout,
    sol1.str_cst.s + sol2.str_cst.s + sol3.str_cst.s - sol_diag.str_cst.s,
    rmax = 2
)

┌─────────────────┬────────────────────────────┬───────────┐
│ Field           │ Structure constant         │ Rel. err. │
├─────────────────┼────────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 0.0+0.0im                  │ 0         │
│ (1, 0)          │ 0                          │ 0.31      │
│ (1, 1)          │ 5.17633e-20-1.52054e-19im  │ 6e-19     │
│ (2, 0)          │ -2.60874e-19-4.11794e-18im │ 4.3e-16   │
│ (2, 1//2)       │ 1.18137e-18+1.02885e-18im  │ 2.7e-16   │
│ (2, 1)          │ -6.71868e-19-5.58104e-19im │ 4e-16     │
└─────────────────┴────────────────────────────┴───────────┘


## $\left\langle (0, \frac12)^3 (2, 0) \right\rangle$

In [16]:
ind = ((0, 1//2), (2, 0), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, fields, parity=1, precision=20);
solve(blocks, Sig(1, 2, 1), even_spin=Channels(false, true, false), rmax=3);

 Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                  1.0+0.0im │            0.0 │
│    (1, 1) │        -0.63854+0.181026im │    2.98082e-17 │
│    (2, 0) │                          0 │       0.880898 │
│ (2, 1//2) │                          0 │       0.970237 │
│    (2, 1) │                          0 │        1.06903 │
│    (3, 0) │ -7.86521e-05-0.000322813im │    9.03894e-12 │
│ (3, 1//3) │   4.37238e-05+0.00014278im │    1.86741e-11 │
│ (3, 2//3) │  4.92031e-05+9.64753e-05im │    1.78644e-11 │
│    (3, 1) │ -8.77471e-05-9.14424e-05im │    9.44656e-12 │
└───────────┴────────────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────────┬────────────────┐
│ Field     │ Structure constant     │ Relative error │
├───────────┼────────────────────────┼────────────────┤
│    (2, 0) │    0.413186+0.31

## $\left\langle (0, \frac12)^3 (3, 0) \right\rangle$

# Polynomials

In [7]:
ind = ((0, 1//2), (0, c.β^2), (0, 1//2), (0, 1//2))
co = Correlation([Field(c, r=r, s=s) for (r, s) in ind], 40)
blocks = precompute_blocks(ind, fields, parity=1, precision=20);
solve(blocks, Sig(0, 0, 0), even_spin=Channels(true), rmax=3);

CompositeException: TaskFailedException

    nested task error: BoundsError: attempt to access 4-element Vector{Dict{Field{Complex{BigFloat}}, Vector{Complex{BigFloat}}}} at index [5]
    Stacktrace:
     [1] throw_boundserror(A::Vector{Dict{Field{Complex{BigFloat}}, Vector{Complex{BigFloat}}}}, I::Tuple{Int64})
       @ Base ./essentials.jl:15
     [2] getindex
       @ ./essentials.jl:919 [inlined]
     [3] macro expansion
       @ ~/Documents/Recherche/projet_these/code/BootstrapVirasoro/src/4_bootstrapsystem.jl:339 [inlined]
     [4] (::BootstrapVirasoro.var"#evaluate_blocks##2#evaluate_blocks##3"{BootstrapVirasoro.var"#evaluate_blocks##4#evaluate_blocks##5"{Vector{LeftRight{BootstrapVirasoro.PosCache}}, Vector{Block{Complex{BigFloat}}}, Vector{Dict{Field{Complex{BigFloat}}, Vector{Complex{BigFloat}}}}, UnitRange{Int64}}})(tid::Int64; onethread::Bool)
       @ BootstrapVirasoro ./threadingconstructs.jl:276
     [5] #evaluate_blocks##2
       @ ./threadingconstructs.jl:243 [inlined]

In [8]:
import BootstrapVirasoro.LoopModels: compute_reference, DoubleGamma,
    residue_ρ

n = -2cospi(c.β^2)
w = 2cospi(2c.β * diag_field[:left].P)
DG = DoubleGamma(c.β)

ds = Dict(
    (1, 0) => 2w,
    (2, 0) => (n^2 - 4) * (w-1) * (w+2),
    (2, 1) => (n^2 - 4) * (w+1) * (w-2),
)

res = Dict(
    (r, s) => 
)

for (r, s) in sort(collect(keys(ds)))
    V = Field(c, r=r, s=s)
    DP = compute_reference(co, diag_field, :s, DG)
    DV = compute_reference(co, V, :s, DG)
    println("D[$r, $s] = $(ds[r, s] * DV / DP)")
end

Base.Meta.ParseError: ParseError:
# Error @ /Users/paulroux/Documents/Recherche/projet_these/code/BootstrapVirasoro/examples/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X54sZmlsZQ==.jl:16:1
    (r, s) => 
)
╙ ── unexpected `)`